links:

https://towardsdatascience.com/how-to-apply-k-means-clustering-to-time-series-data-28d04a8f7da3


https://tslearn.readthedocs.io/en/stable/gettingstarted.html

https://tslearn.readthedocs.io/en/stable/gettingstarted.html

https://tslearn.readthedocs.io/en/stable/gen_modules/neighbors/tslearn.neighbors.KNeighborsTimeSeriesClassifier.html#tslearn.neighbors.KNeighborsTimeSeriesClassifier.score


https://t-dab.medium.com/multivariate-time-series-clustering-e2619552565a

http://alexminnaar.com/2014/04/16/Time-Series-Classification-and-Clustering-with-Python.html

https://datascience.stackexchange.com/questions/29287/multivariate-time-series-clustering

https://towardsdatascience.com/multivariate-time-series-clustering-using-growing-neural-gas-and-spectral-clustering-adc83680639f




In [ ]:
pip install tslearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tslearn-0.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (875 kB)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
from sklearn.model_selection import train_test_split
import os
from tqdm.auto import tqdm
from sklearn import decomposition
import shutil

In [ ]:
import tslearn

In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

### dataset from library (just for trying)

In [ ]:
from tslearn.datasets import UCR_UEA_datasets
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("TwoPatterns")
print(X_train.shape)
# shape : number of objects, time series max length, 1

(1000, 128, 1)


In [ ]:
len(X_train[0])

128

In [ ]:
print(y_train.shape)

## Read data


In [ ]:
# path of original data folder
csvD_path = "/content/drive/MyDrive/Thesis/3rd_semester/Data/csv_files"
demographics = "/content/drive/MyDrive/Thesis/3rd_semester/Data/demographics.csv"

#### Demographics data

In [ ]:
dem_df = pd.read_csv(demographics)

In [ ]:
dem_df.head()

,ID,Study,Group,Subjnum,Gender,Age,Height (meters),Weight (kg),HoehnYahr,UPDRS,UPDRSM,TUAG,Speed_01 (m/sec),Speed_10
0,GaPt03,Ga,PD,3,female,82,1.45,50.0,3.0,20.0,10.0,36.34,NaN,0.778
1,GaPt04,Ga,PD,4,male,68,1.71,NaN,2.5,25.0,8.0,11.00,0.642,0.818
2,GaPt05,Ga,PD,5,female,82,1.53,51.0,2.5,24.0,5.0,14.50,0.908,0.614
3,GaPt06,Ga,PD,6,male,72,1.70,82.0,2.0,16.0,13.0,10.47,0.848,0.937
4,GaPt07,Ga,PD,7,female,53,1.67,54.0,3.0,44.0,22.0,18.34,0.677,0.579


#### Data for training the model - choosing total right force as x 

In [ ]:
total_right_features=[None]*306 #306 is the numbe of walks we have
y_df = pd.DataFrame(columns=['ID', 'y', 'level'])
for i,name in enumerate(os.listdir(csvD_path)):
  id = name.split('_')[0]
  level = dem_df['HoehnYahr'][dem_df['ID'] == id ].values[0]
  if 'Co' in name:
    parkinson = 0
  elif 'Pt' in name:
    parkinson = 1
  y_df = y_df.append({'ID': id, 'y': parkinson, 'level': level}, ignore_index=True)
  total_right_features[i] = pd.read_csv(csvD_path+'/' + name).iloc[:1000, 18].values
  

In [ ]:
# null values are assigned for control group so we replace it with 0
y_df['level'].fillna(0, inplace = True)

class labels are 0,1,2,2.5,3
Converting them to : 0,1,2,3,4 for classifing

In [ ]:
y_df['level'].replace(3, 4, inplace= True)

In [ ]:
y_df['level'].replace(2.5, 3, inplace= True)

In [ ]:
y=y_df['level'].astype('int')

In [ ]:
y_binary = y_df['y'].astype('int')

#### train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_right_features, y, test_size=0.33, random_state=42)

train test split - binary

In [ ]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(total_right_features, y_binary, test_size=0.33, random_state=42)

### KNN classifier (univariate: class: level of Parkinson disease : 0,1,2,3,4)

In [ ]:
clf = KNeighborsTimeSeriesClassifier( metric="dtw")

In [ ]:
clf.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [ ]:
clf.score(X_test, y_test)

0.38613861386138615

### KNN Classifier (univeriate, class : control/PD: 0/1)

In [ ]:
clf_b = KNeighborsTimeSeriesClassifier( metric="dtw")
clf_b.fit(X_train_b, y_train_b)
clf_b.score(X_test_b, y_test_b)

0.7128712871287128

### Multivariate classifier

#### trying how to pass multivariate dataset

In [ ]:
c = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="dtw")
c.fit([[[1, 2, 3],[1, 2, 3]] , [[1, 1.2, 3.2],[1, 2, 3]], [[3, 2, 1],[1, 2, 3]]],y=[0, 0, 1]).predict([[[1, 2.2, 3.5],[1, 2, 3]]]) 

array([0])

Reading our multivariate dataset

In [ ]:
features=[None]*306 #306 is the numbe of walks we have
y_df = pd.DataFrame(columns=['ID', 'y', 'level'])
for i,name in enumerate(os.listdir(csvD_path)):
  id = name.split('_')[0]
  level = dem_df['HoehnYahr'][dem_df['ID'] == id ].values[0]
  if 'Co' in name:
    parkinson = 0
  elif 'Pt' in name:
    parkinson = 1
  y_df = y_df.append({'ID': id, 'y': parkinson, 'level': level}, ignore_index=True)
  features[i] = pd.read_csv(csvD_path+'/' + name).iloc[:1000, 1:15].values # each element of features is a list of list = list of rows 

In [ ]:
y_binary = y_df['y'].astype('int')

In [ ]:
len(features[0]) # (306,1000,14)

1000

In [ ]:
len(features[0][0])

14

In [ ]:
 # checking the dimentions of x data
len(X_train[0])

1000

#### Multivariate classifier - PG/Control classification

In [ ]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(features, y_binary, test_size=0.33, random_state=42)

In [ ]:
clf_b = KNeighborsTimeSeriesClassifier( metric="dtw")
clf_b.fit(X_train_b, y_train_b)
clf_b.score(X_test_b, y_test_b)

0.8217821782178217

### Multivariate Classifier (class labels: parkinson's level)

In [ ]:
# null values are assigned for control group so we replace it with 0
y_df['level'].fillna(0, inplace = True)
y_df['level'].replace(3, 4, inplace= True)
y_df['level'].replace(2.5, 3, inplace= True)
y=y_df['level'].astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_right_features, y, test_size=0.33, random_state=42)

In [ ]:
clf = KNeighborsTimeSeriesClassifier( metric="dtw")
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.38613861386138615